## MISSING VALUES REMOVING METHODS 

### METHOD 1 (USING MEAN,MEDIAN,MODE)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# load the Titanic dataset
data = sns.load_dataset('titanic')
# check the number of missing values in each column
data.isnull().sum().sort_values(ascending=False)
# Drop deck column 
data=data.drop("deck",axis=1)
data.head()
# impute missing values with mean
data['age'] = data['age'].fillna(data['age'].mean())
# impute missing values with median
data['embark_town'] = data['embark_town'].fillna(data['embark_town'].mode()[0])
# impute missing values with mode
data['embarked'] = data['embarked'].fillna(data['embarked'].mode()[0])

# check the number of missing values in each column
data.isnull().sum().sort_values(ascending=False)

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

### METHOD 2 (K-Nearest Neighbors (KNN))
#### FOR ONE COLUMN(AGE)
##### USES SIMILAR ROWS TO FILL MISSING VALUES

In [5]:
# load the dataset
df = sns.load_dataset('titanic')
# check the number of missing values in each column
df.isnull().sum().sort_values(ascending=False)

# impute missing values with KNN imputer
from sklearn.impute import KNNImputer
# call the KNN class with number of neighbors = 4
imputer = KNNImputer(n_neighbors=4)

#impute missing values with KNN imputer
data['age'] = imputer.fit_transform(data[['age']])
# check the number of missing values in each column
data.isnull().sum().sort_values(ascending=False)

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

### METHOD 3 (REGRESSION IMPUTATION)
##### PREDICT MISSING VALUES USING OTHER FEATURES OR VARIABLE.

In [6]:
# load the dataset
df = sns.load_dataset('titanic')
# check the number of missing values in each column
df.isnull().sum().sort_values(ascending=False)

# impute missing values with regression imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# call the IterativeImputer class with max_iter = 10
imputer = IterativeImputer(max_iter=10)

#impute missing values with regression imputer
df['age'] = imputer.fit_transform(df[['age']])

# check the number of missing values in each column
df.isnull().sum().sort_values(ascending=False)

deck           688
embarked         2
embark_town      2
survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
class            0
who              0
adult_male       0
alive            0
alone            0
dtype: int64

### METHOD 4 (MICE)

In [7]:
# imoprt libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# laod the dataset
df = sns.load_dataset('titanic')
df.head()
# check the missing values
df.isnull().sum().sort_values(ascending=False)

from sklearn.preprocessing import LabelEncoder

# create a LabelEncoder object using LabelEncoder() in for loop for categorical columns
# Columns to encode
columns_to_encode = ['sex', 'embarked', 'who', 'deck', 'class', 'embark_town', 'alive']

# Dictionary to store LabelEncoders for each column
label_encoders = {}

# Loop to apply LabelEncoder to each column for encoding
for col in columns_to_encode:
    # Create a new LabelEncoder for the column
    le = LabelEncoder()
    # Fit and transform the data
    df[col] = le.fit_transform(df[col])
    # Store the encoder in the dictionary
    label_encoders[col] = le
df.head()

# impute the missing values with IterativeImputer
# call the IterativeImputer class with max_iter = 10
imputer = IterativeImputer(max_iter=10)

#impute missing values using IterativeImputer in a for loop for age, embark_town,embarked columns and deck

# Columns to impute
columns_to_impute = ['age', 'embark_town', 'embarked', 'deck']

# Loop to impute each column
for col in columns_to_impute:
    df[col] = imputer.fit_transform(df[[col]])    
# check the missing values
df.isnull().sum().sort_values(ascending=False)

# Inverse transform for encoded columns
for col in columns_to_encode:
    # Retrieve the corresponding LabelEncoder for the column
    le = label_encoders[col]
    # Inverse transform the data and convert to integer type
    df[col] = le.inverse_transform(df[col].astype(int))
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### METHOD 5 (RANDOM FOREST IMPUTATION)
##### it handle missing values inherently. They can also be used to predict missing values based on the patterns learned from the other data.

In [9]:
# 1. IMPORT LIBRARIES
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


# 2. LOAD & BASIC CLEANING
df = sns.load_dataset('titanic')

# Drop column with too many missing values
df.drop('deck', axis=1, inplace=True)

# 3. CATEGORICAL ENCODING
categorical_cols = ['sex', 'embarked', 'who', 'class', 'embark_town', 'alive']

label_encoders = {}

for col in categorical_cols:
    df[col] = df[col].astype(str)  # Handle NaN safely
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 4. SPLIT DATA FOR AGE IMPUTATION
df_missing_age = df[df['age'].isna()]
df_known_age = df.dropna(subset=['age'])

X = df_known_age.drop('age', axis=1)
y = df_known_age['age']

# 5. TRAIN-TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 6. RANDOM FOREST MODEL
rf_model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

rf_model.fit(X_train, y_train)

# 7. MODEL EVALUATION
y_pred = rf_model.predict(X_test)

print("Random Forest Age Imputation Performance")
print("----------------------------------------")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

# 8. IMPUTE MISSING AGE VALUES
df_missing_age['age'] = rf_model.predict(
    df_missing_age.drop('age', axis=1)
)

# 9. COMBINE DATA BACK
df_complete = pd.concat([df_missing_age, df_known_age])

# 10. INVERSE TRANSFORM CATEGORIES
for col in categorical_cols:
    le = label_encoders[col]
    df_complete[col] = le.inverse_transform(df_complete[col])

# 11. FINAL CHECK
print("\nMissing values after imputation:")
print(df_complete.isnull().sum())

print("\nFinal Dataset Shape:", df_complete.shape)
print(df_complete.head())


Random Forest Age Imputation Performance
----------------------------------------
RMSE: 11.081260589808045
R2 Score: 0.33769388288226154
MAE: 8.666661815622195

Missing values after imputation:
survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

Final Dataset Shape: (891, 14)
    survived  pclass     sex        age  sibsp  parch     fare embarked  \
5          0       3    male  32.976583      0      0   8.4583        Q   
17         1       2    male  35.642218      0      0  13.0000        S   
19         1       3  female  18.347000      0      0   7.2250        C   
26         0       3    male  35.571486      0      0   7.2250        C   
28         1       3  female  20.651429      0      0   7.8792        Q   

     class    who  adult_male  embark_town alive  alone  
5    Third    

C:\Users\CC\AppData\Local\Temp\ipykernel_11004\3374109681.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_missing_age['age'] = rf_model.predict(
